In [1]:
import numpy as np
import pickle
import random
import json
import os

In [8]:
# load generated attention maps
att_1 = np.load('boards/pythia_cycle_consistent/3014/gq_50000.npy',allow_pickle=True).item()
att_2 = np.load('boards/pythia_cycle_consistent/3014/gq_60000.npy',allow_pickle=True).item()

att_3 = np.load('boards/default/30/gq_50000.npy',allow_pickle=True).item()
att_4 = np.load('boards/default/30/gq_60000.npy',allow_pickle=True).item()

In [3]:
# map implications to its question id

map_12 = {}
    
for i,a in enumerate(att_2['ques_answers']):
    qid = int(a['ques_id']/10)
    if qid not in map_12:
        map_12[qid] = [i]
    else:
        map_12[qid].append(i)
        
map_34 = {}

for i,a in enumerate(att_4['ques_answers']):
    qid = int(a['ques_id']/10)
    if qid not in map_34:
        map_34[qid] = [i]
    else:
        map_34[qid].append(i)

In [4]:
def bd(a1,a2):
    return ((a1 - a2)**2).mean()

In [5]:
dist = 0
count = 0
for a1 in att_1['ques_answers']:
    qid = a1['ques_id']
    if qid in map_12:
        for idx in map_12[qid]:
            a2 = att_2['ques_answers'][idx]
            if qid*10+1 == a2['ques_id']:
                ori = a1['attention']
                le = a2['attention']
                dist += bd(ori,le)
                count +=1

print(dist/count)
        
dist = 0
count = 0
for a1 in att_3['ques_answers']:
    qid = a1['ques_id']
    if qid in map_34:
        for idx in map_34[qid]:
            a2 = att_4['ques_answers'][idx]
            if qid*10+1 == a2['ques_id']:
                ori = a1['attention']
                le = a2['attention']
                dist += bd(ori,le)
                count +=1

print(dist/count)

0.0005833019124185433
0.0011412140758606032


In [6]:
print(count)

119697


In [78]:
imdb = numpy.load('data/imdb_imps/imdb_train2014.npy',allow_pickle=True)

In [79]:
from scipy.stats import mode

imdb_just_imps = [imdb[0].copy()]
_map = {'[1, 0, 0]':1,'[0, 1, 0]':2,'[0, 0, 1]':3}
for i in imdb[1:]:
    
    if i['is_imps']:
        vans = mode(i['valid_answers'])[0][0]

        for q,a,t in zip(i['qa_tokens'][vans],i['qa_answers'][vans],i['imp_type'][vans]):
            cp = i.copy()
            cp.pop('qa_tokens',None)
            cp.pop('qa_answers',None)
            cp.pop('is_imps',None)
            cp.pop('imp_type',None)
            cp['question_id'] = i['question_id']*10+_map[str(t)]
            cp['question_str'] = ' '.join(q)
            cp['question_tokens'] = q
            cp['valid_answers'] = [a for _ in range(10)]
            cp['all_answers'] = [a for _ in range(10)]
            imdb_just_imps.append(cp)
    

In [80]:
pickle.dump(imdb_just_imps,open('data/imdb_just_imps/imdb_train2014.npy','wb'))

In [7]:
att_3['ques_answers'][6]['attention'].shape

(100, 1)